In [1]:
from datetime import datetime

from tensorflow import keras
import tensorflow as tf
import numpy as np

from OperatorLayer import SymmetricOperator
from NormalizedMeanSquaredError import NormalizedMeanSquaredError

In [2]:
data_name = 'Duffing_Equation_expt4'  ## FILL IN HERE (from file name)
data_folder = '../NODE-Operators/data/'

# data is num_steps x num_examples x n
data_train_u = np.load(data_folder + "{}_train1_u.npy".format(data_name))
data_train_f = np.load(data_folder + "{}_train1_f.npy".format(data_name))

# data is num_steps x num_examples x n
data_val_u = np.load(data_folder + "{}_val_u.npy".format(data_name))
data_val_f = np.load(data_folder + "{}_val_f.npy".format(data_name))

# data is num_steps x num_examples x n
data_test_u = np.load(data_folder + "{}_test2_u.npy".format(data_name))
data_test_f = np.load(data_folder + "{}_test2_f.npy".format(data_name))

In [3]:
# %load_ext tensorboard

def construct_encoder(n, l, act_layer_config, lin_layer_config):
    # First build the encoder
    input_ = keras.layers.Input(shape=n)
    hidden1 = keras.layers.Dense(n, **act_layer_config)(input_)
    hidden2 = keras.layers.Dense(n, **act_layer_config)(hidden1)
    hidden3 = keras.layers.Dense(n, **lin_layer_config)(hidden2)
    added = keras.layers.Add()([input_, hidden3])
    latentspace = keras.layers.Dense(l, **lin_layer_config)(added)
    encoder = keras.Model(inputs=[input_], outputs=[latentspace])
    return encoder

def construct_decoder(n, l, act_layer_config, lin_layer_config):
    # Now the decoder
    latent_ = keras.layers.Input(shape=l)
    hidden4 = keras.layers.Dense(n, **lin_layer_config)(latent_)
    hidden5 = keras.layers.Dense(n, **act_layer_config)(hidden4)
    hidden6 = keras.layers.Dense(n, **act_layer_config)(hidden5)
    hidden7 = keras.layers.Dense(n, **act_layer_config)(hidden6)
    added_ = keras.layers.Add()([hidden4, hidden7])
    decoder = keras.Model(inputs=[latent_], outputs=[added_])
    return decoder


In [4]:
_, n = data_train_u.shape
l = 20

print("Training contains:", data_train_u.shape[0], "samples.")
print("Validation contains:", data_val_u.shape[0], "samples.")
print("Input vector is", n, "neurons and latent space is", l, "neurons.")

Training contains: 4798 samples.
Validation contains: 1200 samples.
Input vector is 128 neurons and latent space is 20 neurons.


In [5]:
# Set the configuration to be used for layers with activation functions and linear, non-activated functions
#act_layer = dict(activation="relu", kernel_initializer='he_normal')
act_layer = dict(activation="elu", kernel_initializer='he_normal')
lin_layer = dict(activation=None)

# Encoder and decoder for u
u_enc = construct_encoder(n, l, act_layer, lin_layer)
u_dec = construct_decoder(n, l, act_layer, lin_layer)

# Encoder and decoder for u
f_enc = construct_encoder(n, l, act_layer, lin_layer)
f_dec = construct_decoder(n, l, act_layer, lin_layer)

In [6]:
# Set the optimizer to be used
optimizer = keras.optimizers.SGD(lr=0.01)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

# Specify fit options
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

cbs = [keras.callbacks.ModelCheckpoint("dae.h5", save_best_only=True),
       keras.callbacks.EarlyStopping(),
       keras.callbacks.TensorBoard(log_dir=logdir)]

fit_options = dict(batch_size = 20, epochs = 5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
encoder_loss = NormalizedMeanSquaredError()

In [7]:
# Now assemble the COMPLETE, LINKED autoencoder!!
u_input = keras.layers.Input(shape=n)
u_encoded = u_enc(u_input)

f_input = keras.layers.Input(shape=n)
f_encoded = f_enc(f_input)

Operator = SymmetricOperator()
OperatorLayer = Operator(u_encoded)

DiffLayer = keras.layers.Subtract()([OperatorLayer, f_encoded])

u_decoded = u_dec(u_encoded)
f_decoded = f_dec(f_encoded)

#Lv_decoded = f_dec(OperatorLayer)
#Linvf_decoded = u_dec(HypotheticalInverseOperatorLayer)

# Add the Lv=f loss functions
#Lv_decoded.add_loss(NormalizedMeanSquaredError(), OperatorLayer, f_encoded)

linked_aec = keras.Model(inputs = [u_input, f_input], 
                         outputs = [u_decoded, f_decoded, DiffLayer])

In [8]:
#linked_aec.compile(loss=["mse","mse", "mse"], optimizer=optimizer)
linked_aec.compile(loss=[encoder_loss, encoder_loss, "mse"], optimizer=optimizer)

In [9]:
%%time
train_zeros = np.zeros((data_train_u.shape[0], l))
val_zeros = np.zeros((data_val_u.shape[0], l))

val_data = [(data_val_u, data_val_f), 
            (data_val_u, data_val_f, val_zeros)]

hist = linked_aec.fit(x=[data_train_u, data_train_f], 
                      y=[data_train_u, data_train_f, train_zeros], 
                      validation_data=val_data,
                      callbacks=cbs,
                      **fit_options)

Train on 4798 samples, validate on 1200 samples
Epoch 1/5
4798/4798 [==============================] - 2s 339us/sample - loss: 32.9213 - model_1_loss: 0.0720 - model_3_loss: 0.2584 - subtract_loss: 32.5776 - val_loss: 1.3524 - val_model_1_loss: 0.0480 - val_model_3_loss: 0.1932 - val_subtract_loss: 1.1111
Epoch 2/5
4798/4798 [==============================] - 1s 131us/sample - loss: 1.5208 - model_1_loss: 0.0405 - model_3_loss: 0.1727 - subtract_loss: 1.3072 - val_loss: 1.4611 - val_model_1_loss: 0.0368 - val_model_3_loss: 0.1673 - val_subtract_loss: 1.2570
CPU times: user 3.9 s, sys: 1.26 s, total: 5.17 s
Wall time: 2.35 s


In [10]:
L = Operator.get_operator()

# Prove that L = L^T
maxval = tf.reduce_max(L - tf.transpose(L))
minval = tf.reduce_min(L - tf.transpose(L))

print(maxval)
print(minval)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


In [11]:
L

<tf.Tensor: shape=(20, 20), dtype=float32, numpy=
array([[ 9.7085786e-01, -5.8497833e-03,  5.0409157e-03, -2.8856765e-03,
        -9.4164256e-03, -8.7504694e-03, -5.8301375e-03, -3.1823774e-03,
        -2.8548392e-03, -1.4397322e-03, -6.4743902e-03, -8.4049050e-03,
        -7.6888292e-03,  8.1949746e-03, -3.5686686e-03, -4.2094789e-03,
        -2.2930824e-03, -1.3181175e-03,  2.2913425e-03,  5.2124518e-03],
       [-5.8497833e-03,  9.6714008e-01, -6.7858365e-03, -3.5847349e-03,
        -8.6577162e-03, -7.8964410e-03, -3.2776727e-03, -5.4884079e-04,
        -3.1228128e-03, -1.5713392e-03, -6.5794121e-03, -1.0688985e-03,
        -4.1762022e-03, -8.9553927e-05, -1.4480730e-03, -5.5965420e-04,
         4.2241453e-03, -1.3726356e-02, -2.1058025e-03,  2.6914442e-03],
       [ 5.0409157e-03, -6.7858365e-03,  9.8412400e-01,  8.8865060e-04,
         9.5485002e-03,  3.8745902e-03, -7.8534411e-04, -9.6024305e-04,
        -3.3684625e-04,  5.3334609e-04,  1.8248372e-03,  2.4453941e-04,
         1.0